In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def search_sap_help(query):
    query_content = urllib.parse.quote_plus(query)
    url1 = f"https://help.sap.com/http.svc/elasticsearch?area=content&version=&language=en-US&state=PRODUCTION&q={query_content}&transtype=standard,html,pdf,others&product=&to=19&advancedSearch=0&excludeNotSearchable=1"
    url0 = f"https://help.sap.com/http.svc/elasticsearch?area=topproducts&q={query_content}&language=&state=PRODUCTION&transtype=standard,html,pdf,others&product=&to=5&excludeNotSearchable=0"
    response = requests.get(url0)
    if response.status_code == 200:
        responseJson = response.json()
        if responseJson.get('status') == 'OK':
            data = responseJson.get('data', {})
            products = data.get('products', [])
            result_strings = []
            if len(products) != 0:
                for product in products:
                    result_string = product["title"]
                    if product["alternativeProductName"]:
                        result_string += "(Search Term: " + product["alternativeProductName"] + ")"
                    result_strings.append(result_string)
                final_string = "\n".join(result_strings)
                print(final_string)
                return final_string
            else:
                response = requests.get(url1)
                if response.status_code == 200:
                    responseJson = response.json()
                    if responseJson.get('status') == 'OK':
                        data = responseJson.get('data', {})
                        results = data.get('results', [])
                        result_strings = []
                        if len(results) != 0:
                            for result in results:
                                result_string = result["title"]
                                if(result["url"]):
                                    result_url = result.get('url', 'N/A')
                                    result_string += "\n" + "https://help.sap.com" + result_url
                                if result["snippet"]:
                                    snippet = BeautifulSoup(result.get('snippet', 'N/A'), 'html.parser')
                                    result_string += "\n" + snippet.text
                                if result["date"]:
                                    result_string +="\nLast updated: " + result["date"] 
                                result_strings.append(result_string)
                                result_strings.append("\n")
                            final_string = "\n".join(result_strings)
                            print(final_string)
                            return final_string
        else:
            searchResult = "No results found by query from the help.sap.com"
            return searchResult
    else:
        searchResult = f"Request failed with status code {response.status_code}"
        return searchResult

search_sap_help('cosine')

TypeError: can only concatenate list (not "str") to list